In [ ]:
import pandas as pd

from datamender import Mender

main_df = pd.read_csv("https://media.geeksforgeeks.org/wp-content/uploads/nba.csv")
# main_df = pd.read_csv('./expermental/testing_data/titanic.csv')

main_df[].apply()

main_mender = Mender(main_df)

In [ ]:
# Show all the columns
main_df.columns

In [ ]:
# main_mender.setfocus(columns=['Survived', 'Pclass', 'Name'], rows=(0, 50))

In [ ]:
main_mender.setignore(columns=['College'] ,rows=[457])

In [ ]:
main_mender.overview()

In [ ]:
main_mender.insights()

In [ ]:
main_mender.graphs(kind="kde")

In [ ]:

main_mender.setfocus(columns=['Survived'], rows=(1, 50, 2))

In [ ]:
main_mender.show_incomplete()

In [ ]:
from datamender import ImputationStrategies as im_strategy

main_mender.impute_columns(
    columns=["Position__category"],
    strategies=[im_strategy.mean],
    inplace= True
)


In [ ]:
main_mender.column_categories("Position")

In [ ]:
from datamender import EncodingStrategies as en_strategy

category_order_map = {
    "PG": 1,
    "SF": 3,
    "SG": 5,
    "PF": 7,
    "C": 100,
}

main_mender.encode_column(
    column="Position",
    strategy=en_strategy.label_encoding,
    category_order_map=category_order_map
)
